In [5]:
import os
import boto3
from botocore.client import Config

"""Client class to interact with Scaleway Object Storage."""


class ObjectStorageClient:

    region_name = "fr-par"
    endpoint_url = "https://s3.fr-par.scw.cloud"
    bucket_name = "pollution-eau-s3"
    def __init__(self):
        # Need to use V2 signature for upload and V4 for download
        self.client_v2 = self.build_client("s3")
        self.client_v4 = self.build_client("s3v4")

    @staticmethod
    def build_client(signature_version: str = "s3v4"):
        return boto3.session.Session().client(
            service_name='s3',
            config=Config(signature_version=signature_version),
            region_name=ObjectStorageClient.region_name,
            use_ssl=True,
            endpoint_url=ObjectStorageClient.endpoint_url,
            aws_access_key_id=os.getenv('SCW_ACCESS_KEY'),
            aws_secret_access_key=os.getenv('SCW_SECRET_KEY'),
        )

    def list_buckets(self):
        response = self.client_v4.list_buckets()
        return response['Buckets']

    def list_objects(self):
        response = self.client_v4.list_objects(Bucket=self.bucket_name)
        return response['Contents']

    def download_object(self, key, local_path):
        self.client_v4.download_file(self.bucket_name, key, local_path)

    def upload_object(self, local_path, object_name=None):
        if object_name is None:
            object_name = os.path.basename(local_path)
        self.client_v2.upload_file(local_path, self.bucket_name, object_name)

    def delete_object(self, key):
        self.client_v4.delete_object(Bucket=self.bucket_name, Key=key)


In [1]:
import pandas as pd
import boto3
import io
from dotenv import load_dotenv
import os

load_dotenv()

# AWS S3 Configuration
s3_endpoint = "https://s3.fr-par.scw.cloud"  # Replace with your S3-compatible endpoint
access_key = os.getenv("SCW_ACCESS_KEY")  # Replace with your access key
secret_key = os.getenv("SCW_SECRET_KEY")  # Replace with your secret key
bucket_name = "pollution-eau-s3"  # Replace with your bucket name
file_key = "test.csv"  # Key (file name) in the S3 bucket
file_name = "test.csv"

# Connect to S3
s3_client = boto3.client(
    "s3",
    endpoint_url=s3_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    region_name="fr-par",  
)

# Example DataFrame
data = {
    "Name": ["Alice", "Bob", "Charlie"],
    "Age": [25, 30, 35],
    "City": ["New York", "Los Angeles", "Chicago"]
}
df = pd.DataFrame(data)
df.to_csv(file_name, index=False)



response = s3_client.upload_file(file_name, bucket_name, file_key)

response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
csv_data = response["Body"].read().decode("utf-8")

# Load the CSV data into a DataFrame
df_read = pd.read_csv(io.StringIO(csv_data))
print(df_read)

## Attention avec boto3 1.36 le upload file ne fonctionne pas, ce code utilise "boto3==1.34.11"

S3UploadFailedError: Failed to upload test.csv to pollution-eau-s3/test.csv: An error occurred (RequestTimeTooSkewed) when calling the PutObject operation: The difference between the request time and the current time is too large.

In [6]:
storage_client = ObjectStorageClient()

In [ ]:
storage_client.upload_object(test

In [ ]:
response = s3_client.list_objects_v2(Bucket=bucket_name)
response

In [ ]:
# Write the DataFrame to S3 as a CSV
try:
    # Save DataFrame to an in-memory buffer
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer, index=False)

    # Upload the buffer to S3
    s3_client.put_object(Bucket=bucket_name, Key=file_key, Body=csv_buffer.getvalue())
    print(f"DataFrame written to S3 bucket '{bucket_name}' as '{file_key}'.")
except Exception as e:
    print(f"Error writing DataFrame to S3: {e}")

In [ ]:
# Read the CSV file from S3 into a DataFrame
try:
    # Download the CSV file from S3 into an in-memory buffer
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
    csv_data = response["Body"].read().decode("utf-8")

    # Load the CSV data into a DataFrame
    df_read = pd.read_csv(io.StringIO(csv_data))
    print("DataFrame read from S3:")
    print(df_read)
except Exception as e:
    print(f"Error reading DataFrame from S3: {e}")

In [ ]:
AWS_S3_BUCKET = "pollution-eau-s3"
AWS_ACCESS_KEY_ID = os.getenv("SCW_ACCESS_KEY")
AWS_SECRET_ACCESS_KEY = os.getenv("SCW_SECRET_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

df.to_csv(
    f"s3://{AWS_S3_BUCKET}/{file_key}",
    index=False,
    storage_options={
        "key": AWS_ACCESS_KEY_ID,
        "secret": AWS_SECRET_ACCESS_KEY,
    },
)